# **ETL - Variable climáticas**

In [2]:
# [Configuración] Importación de librerías
import pandas as pd
import glob

In [3]:
# Ruta donde están los archivos (ajusta según tu carpeta)
ruta = "../DATA/meteorologica/*.csv"

# Leer todos los CSV en una lista
archivos = glob.glob(ruta)
df_list = [pd.read_csv(f) for f in archivos]

# Concatenar en un solo DataFrame
df = pd.concat(df_list, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30190 entries, 0 to 30189
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CodigoEstacion   30190 non-null  int64  
 1   NombreEstacion   30190 non-null  object 
 2   Variable         30190 non-null  object 
 3   Parametro        30190 non-null  object 
 4   Fecha            30190 non-null  object 
 5   Unidad           30190 non-null  object 
 6   Valor            30190 non-null  float64
 7   NivelAprobacion  30190 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.8+ MB


In [4]:
# Renombrar columnas
nombres_por_indice = {
    0: 'CODIGO',
    1: 'NOMBRE',
    2: 'VARIABLE',
    3: 'PARAMETRO',
    4: 'FECHA',
    5: 'UNIDAD',
    6: 'VALOR',
    7: 'NIVEL APROBACIÓN',
}
df.columns = [nombres_por_indice[i] for i in range(len(df.columns))]

In [5]:
# Renombrar estaciones
estaciones = {
    'NORMAL MANATI  - AUT [29035080]': 'NORMAL MANATI',
    'JUAN DE ACOSTA [29045110]': 'JUAN DE ACOSTA',
    'BARRANQUILLA SEDE [29040450]': 'BARRANQUILLA SEDE',
    'HIBACHARO [14010020]': 'HIBACHARO',
    'AEROPUERTO E. CORTISSOZ - AUT [29045190]': 'AEROPUERTO E. CORTISSOZ',
    'CAMPANOS LOS [29040290]': 'CAMPANOS LOS',
    'CAMPO DE LA CRUZ [29040250]': 'CAMPO DE LA CRUZ',
    'LOMA GRANDE [29030270]': 'LOMA GRANDE',
    'MONTEBELLO [29040020]': 'MONTEBELLO',
    'LEÑA [29040200]': 'LEÑA',
    'PIOJO [14010010]': 'PIOJO',
    'CANDELARIA [29040260]': 'CANDELARIA',
    'PINTADA LA [14010100]': 'PINTADA LA',
    'CASA DE BOMBAS [29030410]': 'CASA DE BOMBAS',
    'POLONUEVO [29040080]': 'POLONUEVO'
}
df['NOMBRE'] = df['NOMBRE'].replace(estaciones)

In [6]:
# Renombrar parámetros
params ={
        'Húmeda relativa calculada máxima diaria': 'MAX',
       'Húmeda relativa calculada mínima diaria': 'MIN',
       'Día pluviométrico (convencional)': 'Convencional',
       'Temperatura máxima diaria': 'MAX',
       'Temperatura mínima diaria': 'MIN'
}
df['PARAMETRO'] = df['PARAMETRO'].replace(params)

In [7]:
# Agregar latitud, longitud y altitud
data = [
    ["BARRANQUILLA SEDE",       10.99761111, -74.79672222, 12],
    ["NORMAL MANATI",           10.45358333, -74.95463889, 10],
    ["JUAN DE ACOSTA",          10.83333333, -75.03333333, 20],
    ["HIBACHARO",               10.72188889, -75.14011111, 80],
    ["AEROPUERTO E. CORTISSOZ", 10.91777778, -74.77972222, 14],
    ["CAMPANOS LOS",            10.55777778, -74.97177778, 100],
    ["CAMPO DE LA CRUZ",        10.37663889, -74.88841667, 4],
    ["LOMA GRANDE",             10.43383333, -75.13158333, 15],
    ["MONTEBELLO",              10.7085, -74.90783333, 100],
    ["LEÑA",                    10.52172222, -74.89077778, 45],
    ["PIOJO",                   10.75, -75.08333333, 320],
    ["CANDELARIA",              10.45511111, -74.88688889, 4],
    ["PINTADA LA",              10.95511111, -74.995, 200],
    ["CASA DE BOMBAS",          10.40833333, -75.12722222, 10],
    ["POLONUEVO",               10.779, -74.85791667, 80],
]

geo_df = pd.DataFrame(data, columns=['NOMBRE', 'LATITUD', 'LONGITUD', 'ALTITUD'])

df = df.merge(geo_df, on='NOMBRE', how='left')

In [8]:
# Convertir Fecha a datetime
df["FECHA"] = pd.to_datetime(df["FECHA"], dayfirst=True, errors="coerce")

In [9]:
# Pivotear requiere una columna única para cada combinación de Variable y Parámetro

df["VarParam"] = df["VARIABLE"] + "_" + df["PARAMETRO"]

df_wide = df.pivot_table(
    index=["CODIGO", "NOMBRE", "FECHA", "LATITUD", "LONGITUD", "ALTITUD"],
    columns="VarParam",   
    values="VALOR",
    aggfunc="mean"  
).reset_index()
df_wide = df_wide.sort_values(by="FECHA")
df_wide.head()

VarParam,CODIGO,NOMBRE,FECHA,LATITUD,LONGITUD,ALTITUD,HUM RELATIVA_MAX,HUM RELATIVA_MIN,PRECIPITACION_Convencional,TEMPERATURA_MAX,TEMPERATURA_MIN
2836,29035080,NORMAL MANATI,2016-12-31,10.453583,-74.954639,10,NaN,NaN,NaN,34.0,24.2
13638,29045190,AEROPUERTO E. CORTISSOZ,2016-12-31,10.917778,-74.779722,14,NaN,NaN,0.0,NaN,NaN
8617,29040450,BARRANQUILLA SEDE,2017-01-01,10.997611,-74.796722,12,NaN,NaN,0.0,NaN,NaN
11174,29045110,JUAN DE ACOSTA,2017-01-01,10.833333,-75.033333,20,NaN,NaN,NaN,32.2,23.0
13639,29045190,AEROPUERTO E. CORTISSOZ,2017-01-01,10.917778,-74.779722,14,NaN,NaN,0.0,NaN,NaN


In [10]:
df_wide.to_csv(r"C:\Users\Hp\DENGUE\JBOOK\Mapas\clima_atlantico.csv", index=False)